In [1]:
import pandas as pd
from rdkit import Chem

reading dataset csv file

In [ ]:
data_path = 'datasets/BBBP.csv'
df = pd.read_csv(data_path)
df = df[["name", "smiles", "p_np"]]
df

reading and representing a molecule using rdkit

In [ ]:
from rdkit.Chem import Draw
from rdkit.Chem import AllChem

test_smiles = df["smiles"].iloc[3]
m = Chem.MolFromSmiles(test_smiles)
AllChem.Compute2DCoords(m)
Draw.MolToImage(m)


In [ ]:
!python -m mordred --help

Computing features for a single molecule using mordred

In [ ]:
from mordred import Calculator, descriptors

calc = Calculator(descriptors, ignore_3D=True)
features = calc(m)
len(features)

Computing features for all molecules

In [ ]:
df

In [ ]:
def generate_features(smiles_series):
    calc = Calculator(descriptors, ignore_3D=True)
    df_ = df.copy()
    mol_series = pd.Series([Chem.MolFromSmiles(s) for s in smiles_series])
    df_ = df_[~mol_series.isna()]
    feats = calc.pandas(mol_series)
    return feats


In [ ]:
from utils import generate_features

feats = generate_features(df.copy()["smiles"])

In [ ]:
import time
from mordred import Calculator, descriptors


start = time.time()
calc = Calculator(descriptors, ignore_3D=True)
df_ = df.copy()
df_["mol"] = [Chem.MolFromSmiles(s) for s in df_["smiles"]]
df_ = df_[~df_["mol"].isna()]
feats = calc.pandas(df_["mol"])
print(f"finished in {time.time() - start}")
feats

converting errors to NaN

In [ ]:
import numpy as np

feats = feats.map(lambda x: np.nan if not isinstance(x, (int, float)) else x)
feats

removing columns with all NaN values

In [ ]:
columns = feats.columns[~feats.isna().all(axis=0)]
feats = feats[columns]
feats

train, dev(val), test split

In [ ]:
from sklearn.model_selection import train_test_split

X = feats
y = df_["p_np"]
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.1, stratify=y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.22, stratify=y_train_val, random_state=42)

mean_vals = X_train.mean(axis=0)

# removing columns that their mean are NaN
columns = X_train.columns[~mean_vals.isna()]
X_train = X_train[columns]
X_val = X_val[columns]
X_test = X_test[columns]

mean_vals = X_train.mean(axis=0)
X_train.fillna(mean_vals, inplace=True)
X_val.fillna(mean_vals, inplace=True)
X_test.fillna(mean_vals, inplace=True)

feature_names = list(X_train.columns)

print(y_train.sum() / len(y_train))
print(y_val.sum() / len(y_val))
print(y_test.sum() / len(y_test))

Removing constant columns

In [ ]:
is_constant = (X_train == X_train.iloc[0]).all()
columns = X_train.columns[~is_constant]
X_train = X_train[columns]
X_val = X_val[columns]
X_test = X_test[columns]
mean_vals = mean_vals[columns]
feature_names = list(columns)
X_train

t-SNE plot

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE


# reducing dimensuions to 2
tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(X_train)

# plot
plt.figure(figsize=(8, 6))

# plot class 0
plt.scatter(X_tsne[y_train == 0, 0], X_tsne[y_train == 0, 1], color='r', label="class 0", alpha=0.5)

# plot class 1
plt.scatter(X_tsne[y_train == 1, 0], X_tsne[y_train == 1, 1], color='b', label="class 1", alpha=0.5)

plt.title("t-SNE Visualization of Classes")
plt.xlabel("t-SNE Component 1")
plt.ylabel("t-SNE Component 2")
plt.legend()
plt.show()


Feature selection

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif


n_selected_features = 50
selector = SelectKBest(score_func=f_classif, k=n_selected_features)
selector.fit(X_train, y_train)
support = selector.get_support()
feature_names = X_train.columns[support]

X_train = X_train[feature_names]
X_val = X_val[feature_names]
X_test = X_test[feature_names]
print(feature_names)

model building

In [49]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)

report

In [ ]:
report = classification_report(y_val, y_pred)
print(report)

Saving model

In [51]:
import pickle

model_dict = {"feature_names": feature_names,
              "scaler": scaler,
              "model": model}

with open("model_dict.pkl", "wb") as f:
    pickle.dump(model_dict, f)

In [ ]:
with open("model_dict.pkl", "rb") as f:
    model_dict = pickle.load(f)

model = model_dict["model"]
model